In [1]:
!pip install opencv-python scikit-image pandas matplotlib tensorflow


In [ ]:
# Cloner le repo GitHub
!git clone https://github.com/Ferdaoues-Jaoued/projet-apprentissage-par-renforcement.git
%cd projet-apprentissage-par-renforcement/notebook

In [ ]:
import pandas as pd

info_path = "../dataset/Info.txt"
records = []

with open(info_path, "r") as f:
    next(f)  # sauter l'en-tête
    for line in f:
        parts = line.strip().split()
        refnum, bg = parts[0], parts[1]
        if parts[2] == "NORM":
            abnormality, severity = "NORM", "N"
            x = y = radius = None
        else:
            abnormality, severity = parts[2], parts[3]
            if len(parts) >= 7:
                x, y, radius = int(parts[4]), int(parts[5]), int(parts[6])
            else:
                x = y = radius = None
        records.append({
            "REFNUM": refnum,
            "BG": bg,
            "CLASS": abnormality,
            "SEVERITY": severity,
            "X": x,
            "Y": y,
            "RADIUS": radius
        })

df = pd.DataFrame(records)

# Labels pour ML
label_map = {"N": 0, "B": 1, "M": 2}
df["label"] = df["SEVERITY"].map(label_map)
df["label_name"] = df["label"].map({0:"Normal",1:"Benign",2:"Malignant"})

df.to_csv("../dataset/info_labeled.csv", index=False)
print(df.head())


   REFNUM BG CLASS SEVERITY      X      Y  RADIUS  label label_name
0  mdb001  G  CIRC        B  535.0  425.0   197.0      1     Benign
1  mdb002  G  CIRC        B  522.0  280.0    69.0      1     Benign
2  mdb003  D  NORM        N    NaN    NaN     NaN      0     Normal
3  mdb004  D  NORM        N    NaN    NaN     NaN      0     Normal
4  mdb005  F  CIRC        B  477.0  133.0    30.0      1     Benign


In [ ]:
import cv2
import os
import numpy as np
import pandas as pd

all_images_dir = "../dataset/images"
patch_dir = "../dataset/patches"
os.makedirs(patch_dir, exist_ok=True)

patch_size = 64

for idx, row in df.iterrows():

    img_path = os.path.join(all_images_dir, row["REFNUM"] + ".pgm")
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        continue

    # 🔹 Cas Benign / Malignant (ROI connue)
    if pd.notna(row["X"]) and pd.notna(row["Y"]) and pd.notna(row["RADIUS"]):

        x_center = int(row["X"])
        y_center = int(row["Y"])
        r = int(row["RADIUS"])

        # sécuriser la zone
        x_start = max(0, x_center - r)
        y_start = max(0, y_center - r)

        x_end = min(img.shape[1], x_center + r)
        y_end = min(img.shape[0], y_center + r)

        patch = img[y_start:y_end, x_start:x_end]

    # 🔹 Cas Normal (pas de ROI)
    else:
        patch = img

    # 🔹 Vérification anti-patch noir
    if patch.size == 0 or patch.mean() < 5:
        continue

    # 🔹 Resize final
    patch_resized = cv2.resize(patch, (patch_size, patch_size))

    patch_name = f"{row['REFNUM']}_{row['label_name']}.png"
    cv2.imwrite(os.path.join(patch_dir, patch_name), patch_resized)

print("✅ Extraction des patches terminée sans erreurs")



✅ Extraction des patches terminée sans erreurs


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

aug_dir = "../dataset/patches_augmented"
os.makedirs(aug_dir, exist_ok=True)

datagen = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

def augment_patch(img_path, n_aug=15):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = img.astype(np.float32)/255.0
    img = np.expand_dims(img, axis=(0,-1))
    gen = datagen.flow(img, batch_size=1)
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    for i in range(n_aug):
        batch = next(gen)
        aug_img = (batch[0].squeeze()*255).astype(np.uint8)
        cv2.imwrite(os.path.join(aug_dir, f"{base_name}_aug_{i}.png"), aug_img)

# Appliquer aux classes minoritaires
for f in os.listdir(patch_dir):
    if "Benign" in f or "Malignant" in f:
        augment_patch(os.path.join(patch_dir,f), n_aug=15)


In [12]:
records = []

for folder in [patch_dir, aug_dir]:
    for f in os.listdir(folder):
        if f.endswith(".png"):
            if "norm" in f.lower():
                label = 0; label_name = "Normal"
            elif "Benign" in f:
                label = 1; label_name = "Benign"
            elif "Malignant" in f:
                label = 2; label_name = "Malignant"
            else:
                continue

            records.append({
                "patch_name": f,
                "label": label,
                "label_name": label_name,
                "folder": folder
            })

patch_df_final = pd.DataFrame(records)
print(patch_df_final['label_name'].value_counts())



label_name
Benign       976
Malignant    832
Normal       207
Name: count, dtype: int64


In [13]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(patch_df_final, test_size=0.2, stratify=patch_df_final['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)


In [ ]:
import shutil

base_dir = "../dataset/dataset_patches"
os.makedirs(base_dir, exist_ok=True)

def copy_patches(df, split_name):
    for idx, row in df.iterrows():
        dst_dir = os.path.join(base_dir, split_name, row['label_name'])
        os.makedirs(dst_dir, exist_ok=True)
        shutil.copy(os.path.join(row['folder'], row['patch_name']), os.path.join(dst_dir, row['patch_name']))

copy_patches(train_df, "train")
copy_patches(val_df, "val")
copy_patches(test_df, "test")


In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    os.path.join(base_dir,"train"),
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical'
)

val_gen = datagen.flow_from_directory(
    os.path.join(base_dir,"val"),
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical'
)
test_gen = datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical',

)



Found 1612 images belonging to 3 classes.
Found 201 images belonging to 3 classes.
Found 202 images belonging to 3 classes.


In [30]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Backbone VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64,64,3))  # augmenter taille pour meilleur feature

x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(3, activation='softmax')(x)

model_vgg = Model(inputs=base_model.input, outputs=output)

# Geler toutes les couches sauf les 4 dernières
for layer in base_model.layers[:-4]:
    layer.trainable = False
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Compilation
model_vgg.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model_vgg.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 64, 64, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 8, 8, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,977,347 (57.13 MB)

 Trainable params: 7,342,083 (28.01 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

In [31]:
history = model_vgg.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30
)


Epoch 1/30
51/51 ━━━━━━━━━━━━━━━━━━━━ 126s 2s/step - accuracy: 0.4066 - loss: 1.2201 - val_accuracy: 0.6517 - val_loss: 0.6970
Epoch 2/30
51/51 ━━━━━━━━━━━━━━━━━━━━ 119s 2s/step - accuracy: 0.5780 - loss: 0.7496 - val_accuracy: 0.6517 - val_loss: 0.6307
Epoch 3/30
51/51 ━━━━━━━━━━━━━━━━━━━━ 123s 2s/step - accuracy: 0.6252 - loss: 0.6592 - val_accuracy: 0.7214 - val_loss: 0.5928
Epoch 4/30
51/51 ━━━━━━━━━━━━━━━━━━━━ 121s 2s/step - accuracy: 0.6791 - loss: 0.6025 - val_accuracy: 0.7761 - val_loss: 0.5764
Epoch 5/30
51/51 ━━━━━━━━━━━━━━━━━━━━ 120s 2s/step - accuracy: 0.6906 - loss: 0.5854 - val_accuracy: 0.7164 - val_loss: 0.5826
Epoch 6/30
51/51 ━━━━━━━━━━━━━━━━━━━━ 119s 2s/step - accuracy: 0.6712 - loss: 0.5805 - val_accuracy: 0.7463 - val_loss: 0.5428
Epoch 7/30
51/51 ━━━━━━━━━━━━━━━━━━━━ 118s 2s/step - accuracy: 0.7105 - loss: 0.5512 - val_accuracy: 0.7811 - val_loss: 0.5170
Epoch 8/30
51/51 ━━━━━━━━━━━━━━━━━━━━ 117s 2s/step - accuracy: 0.7273 - loss: 0.5320 - val_accuracy: 0.7960 - v

In [32]:
test_loss, test_acc = model_vgg.evaluate(test_gen)

print(f"Test loss     : {test_loss:.4f}")
print(f"Test accuracy : {test_acc:.4f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.8786 - loss: 0.3288
Test loss     : 0.3644
Test accuracy : 0.8614
